In [1]:
DATE = "2022-01-31"
TASK = "coronavirus-metapangenome-graph"
DIR = mkpath("$(homedir())/workspace/$(DATE)-$(TASK)")
cd(DIR)

In [36]:
pkgs = [
    "Graphs",
    "MetaGraphs",
    "FileIO",
    "HTTP",
    "JLD2",
    "DataStructures",
    "BioSequences",
    "DataFrames",
    "Random",
    "FASTX",
    "Revise",
    "StatsBase",
    "uCSV",
    "ProgressMeter"
]

import Pkg
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $(basename(pkg))"))
end

import Mycelia

   Resolving package versions...
  No Changes to `~/.julia/dev/Mycelia/docs/Project.toml`
  No Changes to `~/.julia/dev/Mycelia/docs/Manifest.toml`


download refseq metadata file

In [3]:
refseq_summary_url = "https://ftp.ncbi.nih.gov/genomes/refseq/assembly_summary_refseq.txt"
# drop first line which is a comment
refseq_summary_table = DataFrames.DataFrame(uCSV.read(IOBuffer(HTTP.get(refseq_summary_url).body), comment = "#  ", header=1, delim='\t')...)

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category
,String,String,String,String,String
1,GCF_000001215.4,PRJNA164,SAMN02803731,,reference genome
2,GCF_000001405.39,PRJNA168,,,reference genome
3,GCF_000001635.27,PRJNA169,,,reference genome
4,GCF_000001735.4,PRJNA116,SAMN03081427,,reference genome
5,GCF_000001905.1,PRJNA70973,SAMN02953622,AAGU00000000.3,representative genome
6,GCF_000001985.1,PRJNA32665,SAMN02953685,ABAR00000000.1,representative genome
7,GCF_000002035.6,PRJNA13922,SAMN06930106,,reference genome
8,GCF_000002075.1,PRJNA209509,SAMN02953658,AASC00000000.3,representative genome
9,GCF_000002235.5,PRJNA13728,SAMN00829422,AAGJ00000000.6,representative genome


In [4]:
show(refseq_summary_table[1:1, :], allcols=true)

1×23 DataFrame
 Row │ # assembly_accession  bioproject  biosample     wgs_master  refseq_category   taxid  species_taxid  organism_name            infraspecific_name  isolate  version_status  assembly_level  release_type  genome_rep  seq_rel_date  asm_name                submitter                          gbrs_paired_asm  paired_asm_comp  ftp_path                           excluded_from_refseq  relation_to_type_material  asm_not_live_date 
     │ String                String      String        String      String            Int64  Int64          String                   String              String   String          String          String        String      String        String                  String                             String           String           String                             String                String                     String            
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Get the tax_id of all entities in NCBI that are virus or below

In [5]:
# # all viruses
# https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Info&id=10239&lvl=3&lin=f&keep=1&srchmode=1&unlock
# taxid = "10239"
# # coronavirus
# https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?lvl=0&amp;id=11118
taxid = "11118"

"11118"

In [6]:
NEO4J_BIN_DIR = "/home/jupyter-cjprybol/software/neo4j-community-4.4.3/bin"
if !occursin(NEO4J_BIN_DIR, ENV["PATH"])
    ENV["PATH"] = "$(NEO4J_BIN_DIR):" * ENV["PATH"]
end 
DOMAIN = "ncbi-taxonomy.cjp.garden"
# DOMAIN="34.138.169.173"
USERNAME="neo4j"
PASSWORD=readline(joinpath(homedir(), ".config", "neo4j", "ncbi-taxonomy.password.txt"));
DATABASE = "neo4j"
# ADDRESS="neo4j://$(DOMAIN):7687"
ADDRESS="bolt://$(DOMAIN):7687"

"bolt://ncbi-taxonomy.cjp.garden:7687"

In [7]:
# NOTE! *, or 0 distance (e.g. [*0..2]) step range will include source node!!!!
cmd = "MATCH (n)<-[*]-(n2) WHERE n.tax_id IS NOT NULL AND n.tax_id = \"$(taxid)\" RETURN DISTINCT n2.tax_id AS tax_id"
println(cmd)

MATCH (n)<-[*]-(n2) WHERE n.tax_id IS NOT NULL AND n.tax_id = "11118" RETURN DISTINCT n2.tax_id AS tax_id


In [8]:
coronaviridae_tax_ids = readlines(Mycelia.cypher(address = ADDRESS, username = USERNAME, password = PASSWORD, database = DATABASE, cmd=cmd))[2:end]
coronaviridae_tax_ids = strip.(coronaviridae_tax_ids, '"')
coronaviridae_tax_ids = vcat(coronaviridae_tax_ids, taxid)
coronaviridae_tax_ids = parse.(Int, coronaviridae_tax_ids)

1808-element Vector{Int64}:
  693995
 2664420
  681862
  681861
  681883
  681864
  681878
  681870
  681849
  681871
  681880
  681852
  681876
       ⋮
 2078577
 2219863
 2249315
 2707191
 2724194
 2730119
 2050019
 2501930
 2788713
 2830724
 2836890
   11118

In [10]:
refseq_coronaviridae_tax_ids = intersect(coronaviridae_tax_ids, refseq_summary_table[!, "taxid"])

83-element Vector{Int64}:
  694448
 1508220
 2050018
 1508228
 1503291
 2492658
 1241933
 1503292
 1503290
 1384461
   28295
  693999
   11137
       ⋮
  694015
 1159904
 1159908
  572288
 1159902
 1159907
  572289
 1159905
  572290
 1159906
 1159903
 2050019

In [24]:
coronaviridae_refseq_metadata = refseq_summary_table[map(taxid -> taxid in refseq_coronaviridae_tax_ids, refseq_summary_table[!, "taxid"]), :]

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid
,String,String,String,String,String,Int64
1,GCF_000848685.1,PRJNA485481,,,na,28295
2,GCF_000853505.1,PRJNA485481,,,na,11137
3,GCF_000853865.1,PRJNA485481,,,na,277944
4,GCF_000856025.1,PRJNA485481,,,na,11135
5,GCF_000858765.1,PRJNA485481,,,na,290028
6,GCF_000862345.1,PRJNA485481,,,na,11138
7,GCF_000862505.1,PRJNA485481,,,na,11128
8,GCF_000862965.1,PRJNA485481,,,na,11120
9,GCF_000863605.1,,,,na,31631


In [26]:
function ncbi_ftp_path_to_fasta(ftp_path)
    f_name = basename(ftp_path) * "_genomic.fna.gz"
    new_path = joinpath(ftp_path, f_name)
    return new_path
end

ncbi_ftp_path_to_fasta (generic function with 1 method)

In [27]:
function ncbi_ftp_path_to_gff(ftp_path)
    f_name = basename(ftp_path) * "_genomic.gff.gz"
    new_path = joinpath(ftp_path, f_name)
    return new_path
end

ncbi_ftp_path_to_gff (generic function with 1 method)

In [28]:
outdir = mkpath(joinpath(DIR, "ncbi-files"))

"/home/jupyter-cjprybol/workspace/2022-01-31-coronavirus-metapangenome-graph/ncbi-files"

In [37]:
ProgressMeter.@showprogress for row in DataFrames.eachrow(coronaviridae_refseq_metadata)
    if row["taxid"] in coronaviridae_tax_ids
        fasta_url = ncbi_ftp_path_to_fasta(row["ftp_path"])
        fasta_outfile = joinpath(outdir, basename(fasta_url))
        if !isfile(fasta_outfile)
            Downloads.download(fasta_url, fasta_outfile)
        end

        gff_url = ncbi_ftp_path_to_gff(row["ftp_path"])
        gff_outfile = joinpath(outdir, basename(gff_url))
        if !isfile(gff_outfile)
            Downloads.download(gff_url, gff_outfile)
        end
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


In [56]:
max_memory = Int(floor(Sys.free_memory() / 10^9))
min_memory = 1
pantools_jar_path = joinpath(homedir(), "git", "pantools", "dist", "pantools.jar")
pantools = "java -Xms$(min_memory)g -Xmx$(max_memory)g -jar $(pantools_jar_path)"

"java -Xms1g -Xmx10g -jar /home/jupyter-cjprybol/git/pantools/dist/pantools.jar"

In [63]:
ENV["PATH"] = joinpath(homedir(), "software", "neo4j-community-3.5.30", "bin") * ":" * ENV["PATH"]

"/home/jupyter-cjprybol/software/neo4j-community-3.5.30/bin:/home/jupyter-cjprybol/software/neo4j-community-4.4.3/bin:/opt/tljh/user/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin"

In [67]:
# run(`conda activate pantools`)
# run(`conda init bash`)
# conda run -n envname

In [69]:
run(`conda run -n pantools $pantools -h`)


# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/opt/tljh/user/lib/python3.7/site-packages/conda/exceptions.py", line 1078, in __call__
        return func(*args, **kwargs)
      File "/opt/tljh/user/lib/python3.7/site-packages/conda/cli/main.py", line 84, in _main
        exit_code = do_call(args, p)
      File "/opt/tljh/user/lib/python3.7/site-packages/conda/cli/conda_argparse.py", line 82, in do_call
        return getattr(module, func_name)(args, parser)
      File "/opt/tljh/user/lib/python3.7/site-packages/conda/cli/main_run.py", line 23, in execute
        args.dev, args.debug_wrapper_scripts, call)
      File "/opt/tljh/user/lib/python3.7/site-packages/conda/utils.py", line 403, in wrap_subprocess_call
        with Utf8NamedTemporaryFile(mode='w', prefix=tmp_prefix, delete=False) as fh:
      File "/opt/tljh/user/lib/python3.7/site-packages/conda/_vendor/auxlib/compat.py", line 80, in Utf8NamedTemporaryFile
    

LoadError: failed process: Process(`[4mconda[24m [4mrun[24m [4m-n[24m [4mpantools[24m [4m'java -Xms1g -Xmx10g -jar /home/jupyter-cjprybol/git/pantools/dist/pantools.jar'[24m [4m-h[24m`, ProcessExited(1)) [1]


In [ ]:
build a graph database of it's ref sequence, it's annotations, it's DNA, RNA, and AA nodes

nodes:
    type:
    object:
    key1 - value1
    key2 - value2
    
edges:
    use neo4j metagraph model

In [ ]:
need to be able to do this for genbank and fasta + GFF, and do it for a single genome and 3 genomes

In [ ]:
AAmer - 1, 3, 5, 7?
RNAmer - 1, 3, 5, 7, 9, 11, 13, 17, 23, 31, 41, 53
DNAmer - 1, 3, 5, 7, 9, 11, 13, 17, 23, 31, 41, 53